# Анализ вакансий специалистов DA в LinkedIn

**Задача:** Исследовать рынок открытых вакансий специалистов DA в Европе с помощью спарсенных с LinkedIn данных. Результаты исследования собрать в дашборд.

**Описание данных**
- `title` - должность
- `location` - город
- `country` - страна
- `employment_type` - тип занятости
- `company_name` - название компании
- `employee_qty` - количество сотрудников в компании
- `company_field` - сфера деятельности компании
- `skills` - навыки
- `job_description` - описание вакансии
- `applicants` - количество откливкнувшихся

## Загрузка и предобрабокта данных

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
pth1_data = 'linkedin.csv'

# сюда можно ввести ваш адрес к датасету
pth2_data = ''

try:
    data = pd.read_csv(pth1_data)
except FileNotFoundError:
    data = pd.read_csv(pth2_data)
except:
    print('Something is wrong')

In [3]:
display(data.info(), data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            998 non-null    object 
 1   location         998 non-null    object 
 2   country          998 non-null    object 
 3   employment_type  998 non-null    object 
 4   company_name     996 non-null    object 
 5   employee_qty     998 non-null    object 
 6   company_field    998 non-null    object 
 7   skills           998 non-null    object 
 8   job_description  998 non-null    object 
 9   applicants       838 non-null    float64
dtypes: float64(1), object(9)
memory usage: 78.1+ KB


None

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,None,What You Will Achi...,47.0
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN
4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN


In [4]:
# проверим данные на наличие дубликатов
display(data.duplicated().sum())

112

In [5]:
# удалим дубликаты
data = data.drop_duplicates().reset_index(drop=True)

In [6]:
# проверим данные на наличие пропусков
display(data.isna().sum())

title                0
location             0
country              0
employment_type      0
company_name         2
employee_qty         0
company_field        0
skills               0
job_description      0
applicants         159
dtype: int64

Краткий анализ пропусков:
- `company_name` - Т.к. названия компаний понадобятся нам для дальнейшего анализа и их пропущено всего два значения, просто удалим эти строки с пропусками из датафрейма.
- `applicants` - Для того, чтобы заполнить пропуски в этом столбце, можно предположить, что пропуски соответствуют отсутствию откликов на вакансию, т.е. равны 0. Но, если это допущение будет неверным, такая замена сильно исказит исходные данные. Поэтому оставим этот столбец без изменений.

In [7]:
# удалим строки с пропущенными company_name
data.dropna(subset=['company_name'], inplace=True)

In [8]:
# посмотрим на уникальные значения признаков, в которых может содержаться лишняя информация
display(data['employee_qty'].unique(), data['company_field'].unique())

array(['11-50 employees', 'not specified', '501-1,000 employees',
       '1,001-5,000 employees', '51-200 employees', '10,001+ employees',
       '1-10 employees', '201-500 employees', '5,001-10,000 employees',
       'Retail Apparel and Fashion',
       'See how you compare to 9 applicants. Try Premium for free',
       'Svein Grande is hiring for this job',
       'See how you compare to 19 applicants. Try Premium for free',
       'See how you compare to 22 applicants. Try Premium for free',
       'Romain GUIHENEUF is hiring for this job',
       'See how you compare to 13 applicants. Try Premium for free',
       'See recent hiring trends for Devonshire Hayes Recruitment Specialists Ltd. Try Premium for free',
       'See how you compare to 10 applicants. Try Premium for free',
       'See how you compare to 4 applicants. Try Premium for free'],
      dtype=object)

array(['Staffing and Recruiting', 'not specified',
       'IT Services and IT Consulting', 'Airlines and Aviation',
       'Medical Equipment Manufacturing',
       'Information Technology & Services',
       'Transportation, Logistics, Supply Chain and Storage',
       'Real Estate', 'Financial Services', 'Entertainment Providers',
       'Software Development', '51-200 employees',
       'Technology, Information and Internet', 'Manufacturing',
       'Human Resources', 'Banking', 'Advertising Services',
       'Public Relations and Communications Services',
       'Packaging and Containers Manufacturing', '11-50 employees',
       'Telecommunications', 'Newspaper Publishing', 'Higher Education',
       'Internet Marketplace Platforms', 'Retail', 'Insurance',
       'Leisure, Travel & Tourism', 'Human Resources Services',
       'Motor Vehicle Manufacturing',
       'Personal Care Product Manufacturing', 'Utilities',
       'International Trade and Development', 'Information Services'

In [9]:
# удалим лишнюю информацию из столбца employee_qty и добавим группирующую составляющую
# для этого напишем соответствующие функции 
def search_employees(name):
    name = str(name).lower()
    part_1 = r'\bemployees\b'
    result = re.search(part_1, name)
    if result:
        result = name
    else:
        result = np.nan 
    return result 

def group_employees(name):
    if name is not np.nan:
        if (re.search(r'10\s', name) or re.search(r'50\s', name)) is not None:
            result = 'small'
        elif (re.search(r'200\s', name) or re.search(r'500\s', name)) is not None:
            result = 'medium'
        elif (re.search(r'1,000\s', name) or re.search(r'5,000\s', name) or re.search(r'10,000\s', name)) is not None:
            result = 'big'
        elif re.search(r'10,001\+', name) is not None:
            result = 'very big'
    else:
        result = np.nan
    return result

In [10]:
# применим наши функции к датафрейму
data['employee_qty'] = data['employee_qty'].apply(search_employees)
data['group_employee'] = data['employee_qty'].apply(group_employees)

In [11]:
a = data['employee_qty'].isna().sum()

print(f"Количество пропусков в столбце с размером компаний: {a}.")

Количество пропусков в столбце с размером компаний: 27.


In [12]:
# удалим лишнюю информацию из столбца company_field
# для этого напишем соответствующую функцию
def search_field(name):
    name = str(name).lower()
    part_1 = r'(\b\d{1,}\b)|(not specified)|(\bhiring\b)'
    result = re.search(part_1, name)
    if result:
        result = np.nan
    else:
        result = name   
    return result

In [13]:
# применим нашу функцию к датафрейму
data['company_field'] = data['company_field'].apply(search_field)

In [14]:
b = data['company_field'].isna().sum()

print(f"Количество пропусков в столбце с областью деятельности компаний: {b}.")

Количество пропусков в столбце с областью деятельности компаний: 83.


In [15]:
# отформатируем столбцы location и country
# для этого напишем соответствующую функцию
def func_format(name):
    name = str(name).lower()
    result = re.sub(r'(\bgreater\b)|(\bmetropolitan\b)|(\barea\b)|(\bregion\b)','', name).strip(' ')
    return result        

In [16]:
# применим нашу функцию к датафрейму
data['location'] = data['location'].apply(func_format)
data['country'] = data['country'].apply(func_format)

In [17]:
# выделим вакансии для аналитиков данных и bi аналитиков
# для этого напишем функцию,принимающую на вход значения столбца applicants и возвращающую названия подходящих вакансий
def DA_BI(name):
    name = str(name).lower()
    part_1 = r'((^(?=.*data)(?=.*\banal).*)|(^(?=.*bi)(?=.*\banal).*)|(^(?=.*business)(?=.*intelligence)(?=.*\banal).*))'
    result = re.search(part_1, name)
    if result:
        result = name
    else:
        result = np.nan    
    return result

In [18]:
# применим нашу функцию к датафрейму
data['title'] = data['title'].apply(DA_BI)

In [19]:
c = data['title'].nunique()

print(f"Количество вакансий для аналитиков данных и bi аналитиков: {c}.")

Количество вакансий для аналитиков данных и bi аналитиков: 243.


In [20]:
# сделаем срез исходного датафрейма, оставив в нем только интересующие нас вакансии
data.dropna(subset=['title'], inplace=True)

In [21]:
# выделим хад-скилы из описания вакансий 
# для этого создадим список с наиболее распространенными хард-скилами аналитиков
hard_skills = (['ai', 'ml', 'r', 'a/b testing', 'actian', 'adobe analytics', 'airflow', 'alooma', 'alteryx', 'apache camel', 
                'apache nifi', 'apache spark', 'api', 'asana', 'aws', 'aws glue', 'azure', 'azure data factory', 'basecamp', 
                'bigquery', 'birst', 'blendo', 'bootstrap', 'c#', 'c++', 'cdata sync', 'cognos', 'computer vision', 'css', 
                'dashboard', 'data studio', 'databricks', 'dataddo', 'datahub', 'datastage', 'dbconvert', 'dbeaver', 
                'deep learning', 'dl/ml', 'docker', 'domo', 'erwin', 'etleap', 'excel', 'fivetran', 'gcp', 'github', 'gitlab', 
                'google analytics', 'google data flow', 'grafana', 'hadoop', 'hana', 'hive', 'html', 'ibm coremetrics', 
                'inetsoft', 'informatica', 'integrate.io', 'iri voracity', 'izenda', 'java', 'java script', 'jenkins', 'jira', 
                'julia', 'k2view', 'kantar', 'keras', 'linux', 'logstash', 'looker', 'lstm', 'luidgi', 'matillion', 'matlab', 
                'metabase', 'microsoft sql', 'microstrategy', 'miro', 'ml flow', 'natural nanguage processing', 'nlp', 
                'omniture', 'oracle business intelligence', 'oracle data integrator', 'panorama', 'pentaho', 'postgresql', 
                'power bi', 'power point', 'python', 'pytorchhevo data', 'qlik sense', 'qlikview', 'querysurge', 'rapidminer', 
                'redash', 'redshift', 'rivery', 'salesforce', 'sap business objects', 'sas', 'sas visual analytics', 'scala', 
                'selenium', 'singer', 'sisense', 'skyvia', 'snowflake', 'spark', 'spotfire', 'spss', 'sql', 'ssis', 
                'statistics', 'statsbot', 'stitch', 'streamsets', 'tableau', 'talend', 'targit', 'tensorflow', 'thoughtspot', 
                'timeseries', 'trello', 'vba', 'webfocus', 'wfh', 'xplenty', 'yellowfin', 'javascript', 'mendix', 
                'adobe launch', 'facebook business manager', 'business objects bi', 'tealium', 'google ads','optimizely', 
                'google optimize', 'hubspot', 'auth0', 'oauth', 'adobe target', 'ga360', 'sa360', 'dv360', 
                'adobe audience manager', 'data factory', 'xtract.io', 'hevo data', 'google tag manager', 'powerbi', 'sap', 
                'microsoft sql server', 'oracle', 'yellowfin', 'cluvio', 'adobe experience platform', 'qlik', 'plotly', 
                'mongodb', 'seaborn', 'matplotlib', 'google sheets', 'confluence', 'google workspace', 'pandas', 'numpy', 
                'scikit-learn', 'pytorch', 'kafka', 'dash', 'streamlit', 'shiny', 'amplitude', 'dune', 'postgre', 'mysql', 
                'clickhouse', 'sem rush', 'semrush', 'fuzzy', 'pyspark', 'cloudera', 'hbase', 'hdfs', 'shell', 'vtom', 'unix', 
                'etl', 'stambia', 'power amc', 'powerquery', 'powerpivot', 'openshift', 'kubernetes', 'openstack', 'ansible', 
                'bitbucket', 'elasticsearch', 'kibana', 'couchbase', 'redis', 'splunk', 'big query', 'spreadsheet', 'retool', 
                'dbt', 'segment', 'mixpanel', 'hightouch', 'dataflow', 'telegraf', 'influxdb', 'chronograf', 'kapacitor', 
                'beats', 's3', 'lambda', 'dynamodb', 'posthog', 'google cloud platform', 'nosql', 'sssr', 't-sql', 'cassandra', 
                'powerpoint', 'data fusion', 'elt', 'data warehouse', 'terraform', 'ci/cd', 'amazon web services', 'raphtory', 
                'neo4j', 'rust', 'julia', 'data mining', 'scipy', 'jmp', 'composer', 'mlflow', 'bash', 'cicd', 'd3.js', 
                'ggplot', 'sparkml', 'amazon machine learning', 'aml', 'mxnet', 'caffe', 'theano', 'cntk', 'conda', 'jupyter', 
                'terraapi', 'octave', 'git', 'svn', 'nltk', 'gensim', 'spacy'])

# и напишем функцию, принимающую на вход значения столбца job_description и возвращающую хард-скилы
def search_hard_skills(name):
    name = str(name).lower()
    result = [word for word in [word.strip(""" !'"#$%&()*+,-./:;<=>?@[\]^_{|}~""") for word in name.split()] 
              if word in hard_skills]
    if len(result) == 0:
        result = np.nan
    else: 
        result = set(result)
    return result

In [22]:
# применим нашу функцию к датафрейму
data['hard_skills'] = data['job_description'].apply(search_hard_skills)

In [23]:
e = data['hard_skills'].isna().sum()

print(f"Количество пропусков в столбце с хард-скилами: {e}.")

Количество пропусков в столбце с хард-скилами: 50.


In [24]:
# удалим из датафрейма столбцы с признаками skills и job_description, которые не понадобятся нам для дальнешйего анализа
data = data.drop(['skills', 'job_description'], axis=1)

In [25]:
# создадим отдельный датафрейм для анализа самых востребованных хард-скилов аналитиков 
hard_skills = data.explode('hard_skills').reset_index(drop=True)

In [26]:
# выгружаем данные для построения дашборда
data.to_csv('linkedin_data_git.csv', encoding='utf8')
hard_skills.to_csv('linkedin_hard_skills_git.csv', encoding='utf8')

## Выводы

### Задача

Исследовать рынок открытых вакансий специалистов DA в Европе с помощью спарсенных с LinkedIn данных. Результаты исследования собрать в дашборд.

### Особенности рынка открытых вакансий DA в Европе

- Самые востребованные хард-скилы для аналитиков в Европе: sql, excel, python, tableau, ai, power bi, r.
- Топ стран по количеству вакансий для аналитиков: Италия, Франция, Великобритания, Германия, Нидерланды.
- Самый распространенный формат работы: гибридный график.

### [Dashboard](https://public.tableau.com/app/profile/narbekovavioletta/viz/dash_linkedin/dash_linkedin)